### 1. Import required modules

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys


from time import sleep
from random import randint
import pickle

In [2]:
option = webdriver.ChromeOptions()
driver = webdriver.Chrome('/Users/audreytang/Desktop/chromedriver')

### 2. Create list of general product pages (e.g. moisturisers, treatments, etc)

In [39]:
## get list of product pages
product_pages={
    'moisturisers':'https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=300', \
    'cleansers':'https://www.sephora.com/shop/cleanser?pageSize=300', \
    'treatments':'https://www.sephora.com/shop/facial-treatments?pageSize=300', \
    'wellness':'https://www.sephora.com/shop/wellness-skincare?pageSize=300', \
    'eyecare':'https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=300', \
    'face_masks':'https://www.sephora.com/shop/face-mask?pageSize=300', \
    'tools':'https://www.sephora.com/shop/skin-care-tools?pageSize=300', \
    'sunscreens':'https://www.sephora.com/shop/sunscreen-sun-protection?pageSize=300', \
    'tanners':'https://www.sephora.com/shop/self-tanning-products?pageSize=300', \
    'lip_treatments':'https://www.sephora.com/shop/lip-treatments?pageSize=300', \
              }


### 3. From general pages, get links to all skincare products

In [ ]:
number_of_products=int(re.findall("\d+",driver.find_elements_by_class_name('css-fgy0ne')[1].find_elements(By.TAG_NAME, "span")[0].text)[0])

# find number of results returned
results_returned=driver.find_elements_by_class_name('css-ix8km1')

# if number of results returned is smaller than number of products available, scroll down to find more
smaller_value=min(300, number_of_products)
while len(results_returned)<smaller_value:
    driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-ix8km1')[-1])
    sleep(1)
    results_returned=driver.find_elements_by_class_name('css-ix8km1')


In [41]:
class Product_Type:
    def __init__(self, product_link):
        self.product_link=product_link
        self.product_links=[]

    def scrolling_product_page(self):
        # find number of results returned
        self.results_returned=driver.find_elements_by_class_name('css-ix8km1')

        # if number of results returned is smaller than number of products available, scroll down to find more
        smaller_value=min(300, self.number_of_products-300*(self.page-1))
        while len(self.results_returned)<smaller_value:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-ix8km1')[-1])
            sleep(1)
            self.results_returned=driver.find_elements_by_class_name('css-ix8km1')
        return
        
    def get_product_links(self):    
        # get all links
        self.scrolling_product_page()
        for i in range(len(self.results_returned)):
            link=self.results_returned[i].get_attribute("href")
            self.product_links.append(link)
        return
    
    def go_to_function(self):
        go_to_page=self.product_link
        driver.get(go_to_page)
        self.page=1

        # find pages required
        # find number of products available
        self.number_of_products=int(re.findall("\d+",driver.find_elements_by_class_name('css-fgy0ne')[1].find_elements(By.TAG_NAME, "span")[0].text)[0])

        # get all product_links
        self.get_product_links()

        # if total number of products > 300, then go on next page, 
        # so number of pages=number_of_products//300 + 1
        number_of_page=self.number_of_products//300 + 1

        if number_of_page>1:
            for page in range(2, number_of_page+1):
                self.page=page
                go_to_page=self.product_link+'&currentPage='+str(self.page)
                driver.get(go_to_page)
                # get all product_links
                self.get_product_links()
        return self.product_links
    


In [42]:
product_links=[]

for product_page in product_pages.values():
    product_type_x=Product_Type(product_page)
    product_links_x=product_type_x.go_to_function()
    product_links=product_links+product_links_x



In [16]:
# save the information we have gathered with pickle

with open('Data Mining from Sephora - Putting It Together', 'wb') as f:
    pickle.dump([product_links, brand_name, product_name,item_id, product_type_1, product_type_2, number_of_reviews, number_of_loves, ratings,price, skin_type_normal, skin_type_dry, skin_type_combination, skin_type_oily, skin_type_sensitive, skin_concerns_fine_lines_wrinkles, skin_concerns_pores, skin_concerns_dark_spots, skin_concerns_firmness_elasticity, skin_concerns_dullness_uneven, skin_concerns_dryness, skin_concerns_oiliness, skin_concerns_puffiness, skin_concerns_redness_irritation, five_star, four_star, three_star, two_star, one_star], f)
    

In [3]:
# with open('Data Mining from Sephora - Putting It Together', 'rb') as f:
#     [product_links, brand_name, product_name,item_id, product_type_1, product_type_2, number_of_reviews, number_of_loves, ratings,price, skin_type_normal, skin_type_dry, skin_type_combination, skin_type_oily, skin_type_sensitive, skin_concerns_fine_lines_wrinkles, skin_concerns_pores, skin_concerns_dark_spots, skin_concerns_firmness_elasticity, skin_concerns_dullness_uneven, skin_concerns_dryness, skin_concerns_oiliness, skin_concerns_puffiness, skin_concerns_redness_irritation, five_star, four_star, three_star, two_star, one_star]= pickle.load(f)


### 4. Go to each product page, and web scrap required info

In [4]:
class Product_Info:
    def __init__(self, starting_value, product_list):
        self.starting_value=starting_value
        self.product_list=product_list
        
        
    def load_selenium_beautifulsoup(self, product_link):
        # load selenium and beautifulsoup
        driver.get(product_link)

        response = requests.get(product_link)
        self.soup = BeautifulSoup(response.text, 'html.parser')

    def scrolling_to_product_type(self):
        while not driver.find_elements_by_class_name('css-dvzm2b')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-dvzm2b')[0])
            sleep(2)
        return
    
        
    def scrolling_to_id(self):
        if not driver.find_elements_by_class_name('css-v7k1z0')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-v7k1z0')[0])
            sleep(2)
        return

    def scrolling_to_price(self):
        if not driver.find_elements_by_class_name('css-slwsq8')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-slwsq8')[0])
            sleep(2)
        return
    
    def scrolling_to_reviews(self):
        while not driver.find_elements_by_class_name('css-fkgzyo') and not driver.find_elements_by_class_name('css-2ijuk8'):
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_element_by_xpath("//div[@id='ratings-reviews']"))
            sleep(3)
            print('signal')
        return
    
    def scrolling_to_skin_type(self):
        if not driver.find_elements_by_class_name('css-pz80c5')[0].text:
            driver.execute_script("return arguments[0].scrollIntoView();", driver.find_elements_by_class_name('css-pz80c5')[0])
            sleep(2)
        return
    
    def substring_in_string(self, substring):  # update substring
        if substring.lower() in self.skin_description.lower():
            return 1
        else:
            return 0
        


    def get_info(self, i, product_link):
        self.load_selenium_beautifulsoup(product_link)
        if driver.current_url=='https://www.sephora.com/search?keyword=productnotcarried':
            return
        
        else:
            # product info
            self.scrolling_to_product_type()
            if len(driver.find_elements_by_class_name('css-dvzm2b'))>1:
                product_type_1[i]=driver.find_elements_by_class_name('css-dvzm2b')[1].text
            
            product_type_2[i]=driver.find_elements_by_class_name('css-lrl8sh')[0].text
            brand_name[i]=driver.find_elements_by_class_name('css-euydo4')[0].text
            product_name[i]=driver.find_elements_by_class_name('css-140z8k4')[0].find_elements(By.TAG_NAME, "span")[1].text

            self.scrolling_to_id()
            item_id[i]=int(re.findall("\d+", driver.find_elements_by_class_name('css-v7k1z0')[0].text)[0])

            self.scrolling_to_price()
            price[i]=float(re.findall(r"[-+]?[0-9]*\.?[0-9]+", driver.find_elements_by_class_name('css-slwsq8')[0].text)[0])
            number_of_loves[i]=int(self.soup.find('div', class_='css-jtf0an').find('span',class_='css-2rg6q7').span.text)


            # skin concerns/ types
            self.scrolling_to_skin_type()
            self.skin_description=driver.find_elements_by_class_name('css-pz80c5')[0].text

            skin_type_normal[i]=self.substring_in_string('Normal')
            skin_type_dry[i]=self.substring_in_string('Dry')
            skin_type_combination[i]=self.substring_in_string('Combination')
            skin_type_oily[i]=self.substring_in_string('Oily')
            skin_type_sensitive[i]=self.substring_in_string('Sensitive')
            skin_concerns_pores[i]=self.substring_in_string('Pores')
            skin_concerns_dark_spots[i]=self.substring_in_string('Dark Spots')
            skin_concerns_firmness_elasticity[i]=self.substring_in_string('Loss of firmness and elasticity')
            skin_concerns_dryness[i]=self.substring_in_string('Dryness')
            skin_concerns_oiliness[i]=self.substring_in_string('Oiliness')
            skin_concerns_puffiness[i]=self.substring_in_string('Puffiness')
            skin_concerns_redness_irritation[i]=self.substring_in_string('Normal')

            if 'dullness' in self.skin_description.lower() or 'uneven' in self.skin_description.lower():
                skin_concerns_dullness_uneven[i]=1
            else:
                skin_concerns_dullness_uneven[i]=0

            if 'fine lines' in self.skin_description.lower() or 'wrinkles' in self.skin_description.lower():
                skin_concerns_fine_lines_wrinkles[i]=1
            else:
                skin_concerns_fine_lines_wrinkles[i]=0

            # review info
            self.scrolling_to_reviews()
            print('signal2')


            if driver.find_elements_by_class_name('css-2ijuk8'):
                print('signal3')
                return
            else:
                number_of_reviews[i]=int(re.findall("\d+", driver.find_elements_by_class_name('css-tc6qfq')[0].text)[0])
                star_votes = driver.find_elements_by_class_name('css-960eb6')[0].find_elements(By.TAG_NAME, "tr") # get all of the rows in the table

                # Get votes for different stars    
                col = star_votes[0].find_elements(By.TAG_NAME, "td")[2].text 
                five_star[i]=int(col)

                col = star_votes[1].find_elements(By.TAG_NAME, "td")[2].text 
                four_star[i]=int(col)

                col = star_votes[2].find_elements(By.TAG_NAME, "td")[2].text 
                three_star[i]=int(col)

                col = star_votes[3].find_elements(By.TAG_NAME, "td")[2].text 
                two_star[i]=int(col)

                col = star_votes[4].find_elements(By.TAG_NAME, "td")[2].text 
                one_star[i]=int(col)

        
    def go_through_product_list(self):
        for i in range(len(self.product_list)):
            try:
                sleep(randint(2,30))
                print(i, self.product_list[i])
                self.get_info(self.starting_value+i, self.product_list[i]) 
                print(five_star[i])
        
            except:
                pass 
        return 





In [14]:
product_list_1=Product_Info(0,product_links[0:]) 
product_list_1.go_through_product_list()

0 https://www.sephora.com/ca/en/product/clean-smart-facial-cleansing-device-P433873?icid2=products%20grid:p433873
signal
signal2
2612
1 https://www.sephora.com/ca/en/product/double-cleansing-method-P411370?icid2=products%20grid:p411370
signal
signal2
1848
2 https://www.sephora.com/ca/en/product/polish-de-puff-duo-P443806?icid2=products%20grid:p443806
signal
signal2
982
3 https://www.sephora.com/ca/en/product/acellerator-ultra-beauty-device-for-face-body-P407883?icid2=products%20grid:p407883
signal
signal2
390
4 https://www.sephora.com/ca/en/product/green-tea-infused-facial-sponge-P414547?icid2=products%20grid:p414547
signal
signal2
548
5 https://www.sephora.com/ca/en/product/personal-microderm-P433874?icid2=products%20grid:p433874
signal
signal2
3454
6 https://www.sephora.com/ca/en/product/silicone-cleaning-spray-P397810?icid2=products%20grid:p397810
signal2
704
7 https://www.sephora.com/ca/en/product/nuface-nuface-fix-wanderlust-collection-P456541?icid2=products%20grid:p456541
signal


signal
signal2
5875
57 https://www.sephora.com/ca/en/product/powerful-wrinkle-reducing-cream-sunscreen-broad-spectrum-spf-30-P422265?icid2=products%20grid:p422265
signal
signal2
129
58 https://www.sephora.com/ca/en/product/facial-fuel-daily-energizing-moisture-treatment-sunscreen-broad-spectrum-spf-15-P439414?icid2=products%20grid:p439414
signal
signal2
738
59 https://www.sephora.com/ca/en/product/anti-aging-moisturizer-broad-spectrum-spf-30-P405211?icid2=products%20grid:p405211
signal
signal2
911
60 https://www.sephora.com/ca/en/product/ultimate-sun-protection-lotion-broad-spectrum-spf-50-wetforce-for-sensitive-skin-children-P405944?icid2=products%20grid:p405944
signal
signal2
309
61 https://www.sephora.com/ca/en/product/stem-cellular-cc-cream-P408943?icid2=products%20grid:p408943
signal
signal2
66
62 https://www.sephora.com/ca/en/product/clean-screen-mineral-spf-30-P448899?icid2=products%20grid:p448899
signal
signal2
368
63 https://www.sephora.com/ca/en/product/skin-perfecting-lotion

101
115 https://www.sephora.com/ca/en/product/the-body-illuminating-self-tan-drops-P424989?icid2=products%20grid:p424989
signal
signal2
363
116 https://www.sephora.com/ca/en/product/self-tan-purity-bronzing-water-face-mist-P432262?icid2=products%20grid:p432262
signal
signal2
512
117 https://www.sephora.com/ca/en/product/glow-clear-color-correcting-self-tanning-mousse-P443829?icid2=products%20grid:p443829
signal
signal2
50
118 https://www.sephora.com/ca/en/product/the-gradual-illuminating-gradual-tan-lotion-P424990?icid2=products%20grid:p424990
signal
signal2
193
119 https://www.sephora.com/ca/en/product/st-tropez-tanning-essentials-double-sided-luxe-tan-applicator-mitt-P456400?icid2=products%20grid:p456400
signal2
194
120 https://www.sephora.com/ca/en/product/the-butter-illuminating-tanning-butter-P424986?icid2=products%20grid:p424986
signal
signal2
1709
121 https://www.sephora.com/ca/en/product/argan-liquid-gold-self-tanning-oil-P395245?icid2=products%20grid:p395245
signal
signal2
138

signal
signal2
389
175 https://www.sephora.com/ca/en/product/tatcha-the-kissu-lip-mask-P453225?icid2=products%20grid:p453225
signal
signal2
132
176 https://www.sephora.com/ca/en/product/kush-lip-balm-P440645?icid2=products%20grid:p440645
signal
signal2
28
177 https://www.sephora.com/ca/en/product/sea-quench-lip-rescue-balm-P405601?icid2=products%20grid:p405601
signal
signal2
800
178 https://www.sephora.com/ca/en/product/lip-balm-scrub-P415438?skuId=2009363&icid2=products%20grid:p415438
signal
signal2
25
179 https://www.sephora.com/ca/en/product/givenchy-le-rose-perfecto-liquid-balm-P457245?icid2=products%20grid:p457245
signal
signal2
133
180 https://www.sephora.com/ca/en/product/summer-fridays-lip-butter-balm-P455936?icid2=products%20grid:p455936
signal
signal2
1261
181 https://www.sephora.com/ca/en/product/glow-recipe-watermelon-glow-lip-pop-P455236?icid2=products%20grid:p455236
signal
signal2
381
182 https://www.sephora.com/ca/en/product/brazilian-kiss-cupua-u-lip-butter-P415747?icid

signal2
259
239 https://www.sephora.com/ca/en/product/multipurpose-superbalm-P428619?icid2=products%20grid:p428619
signal
signal2
4
240 https://www.sephora.com/ca/en/product/sephora-collection-give-me-some-sugar-colorful-gloss-balm-set-P457436?icid2=products%20grid:p457436
signal
signal2
signal3
81
241 https://www.sephora.com/ca/en/product/kush-lip-glaze-P440644?icid2=products%20grid:p440644
signal
signal2
45
242 https://www.sephora.com/ca/en/product/pout-prep-lip-exfoliant-P420230?icid2=products%20grid:p420230
signal2
89
243 https://www.sephora.com/ca/en/product/kiehls-butterstick-lip-treatment-spf-25-P456389?icid2=products%20grid:p456389
signal
signal2
28
244 https://www.sephora.com/ca/en/product/repairwear-intensive-lip-treatment-P122782?icid2=products%20grid:p122782
signal
signal2
126
245 https://www.sephora.com/ca/en/product/squalane-rose-vegan-lip-balm-P448210?icid2=products%20grid:p448210
signal
signal2
659
246 https://www.sephora.com/ca/en/product/hyaluronic-marine-collagen-lip

296 https://www.sephora.com/ca/en/product/smashbox-mindful-5-custom-glow-lip-balm-P454726?icid2=products%20grid:p454726
signal
signal2
4
297 https://www.sephora.com/ca/en/product/lanostick-scrubba-balm-lemonaid-P449621?icid2=products%20grid:p449621
signal
signal2
78
298 https://www.sephora.com/ca/en/product/balmy-days-P411852?icid2=products%20grid:p411852
signal
signal2
19
299 https://www.sephora.com/ca/en/product/tinted-lip-balm-P421447?icid2=products%20grid:p421447
signal
signal2
63
300 https://www.sephora.com/ca/en/product/plumprageous-tm-matte-lip-treatment-P404346?icid2=products%20grid:p404346
signal2
205
301 https://www.sephora.com/ca/en/product/basic-balm-P446926?icid2=products%20grid:p446926
signal
signal2
11
302 https://www.sephora.com/ca/en/product/lip-lock-priming-balm-P435730?icid2=products%20grid:p435730
signal
signal2
1
303 https://www.sephora.com/ca/en/product/balmy-nights-P411851?icid2=products%20grid:p411851
signal
signal2
175
304 https://www.sephora.com/ca/en/product/

In [4]:
# check if the program is getting the right information
brand_name

['DRUNK ELEPHANT',
 'TATCHA',
 'TATCHA',
 'LA MER',
 'SK-II',
 'BELIF',
 'DRUNK ELEPHANT',
 'THE ORDINARY',
 "KIEHL'S SINCE 1851",
 'FRESH',
 'FRESH',
 'DRUNK ELEPHANT',
 'GLOW RECIPE',
 'YOUTH TO THE PEOPLE',
 'THE ORDINARY',
 'FRESH',
 'LA MER',
 'CLINIQUE',
 'BELIF',
 'FARMACY',
 'IT COSMETICS',
 'OLEHENRIKSEN',
 'CHARLOTTE TILBURY',
 'GLOW RECIPE',
 'LANEIGE',
 'YOUTH TO THE PEOPLE',
 'CLINIQUE',
 'GLOW RECIPE',
 'YOUTH TO THE PEOPLE',
 'TATCHA',
 'SK-II',
 None,
 'FRESH',
 'GLOW RECIPE',
 'THE ORDINARY',
 "KIEHL'S SINCE 1851",
 'OLEHENRIKSEN',
 'FRESH',
 'BIOSSANCE',
 'CAUDALIE',
 'TATCHA',
 'OLEHENRIKSEN',
 'FARMACY',
 'TATCHA',
 'DRUNK ELEPHANT',
 'BOBBI BROWN',
 'CLINIQUE',
 'JOSIE MARAN',
 'SK-II',
 'FIRST AID BEAUTY',
 'CAUDALIE',
 'SHISEIDO',
 'SEPHORA COLLECTION',
 'IT COSMETICS',
 'YOUTH TO THE PEOPLE',
 'DR. JART+',
 'FIRST AID BEAUTY',
 'PERRICONE MD',
 'LANEIGE',
 'GLAMGLOW',
 'BIOSSANCE',
 'TARTE',
 'BIOSSANCE',
 'LANEIGE',
 'IT COSMETICS',
 'DRUNK ELEPHANT',
 'ORIGINS

### 5. Save arrays as dataframe and csv file

In [28]:
df = pd.DataFrame([product_links, brand_name, product_name,item_id, product_type_1, product_type_2, number_of_reviews, number_of_loves,price, skin_type_normal, skin_type_dry, skin_type_combination, skin_type_oily, skin_type_sensitive, skin_concerns_fine_lines_wrinkles, skin_concerns_pores, skin_concerns_dark_spots, skin_concerns_firmness_elasticity, skin_concerns_dullness_uneven, skin_concerns_dryness, skin_concerns_oiliness, skin_concerns_puffiness, skin_concerns_redness_irritation, five_star, four_star, three_star, two_star, one_star])
df=df.T
df.columns=['product_links', 'brand_name', 'product_name','item_id', 'product_type_1', 'product_type_2', 'number_of_reviews', 'number_of_loves','price', 'skin_type_normal', 'skin_type_dry', 'skin_type_combination', 'skin_type_oily', 'skin_type_sensitive', 'skin_concerns_fine_lines_wrinkles', 'skin_concerns_pores', 'skin_concerns_dark_spots', 'skin_concerns_firmness_elasticity', 'skin_concerns_dullness_uneven', 'skin_concerns_dryness', 'skin_concerns_oiliness', 'skin_concerns_puffiness', 'skin_concerns_redness_irritation', 'five_star', 'four_star', 'three_star', 'two_star', 'one_star']
df


,product_links,brand_name,product_name,item_id,product_type_1,product_type_2,number_of_reviews,number_of_loves,price,skin_type_normal,...,skin_concerns_dullness_uneven,skin_concerns_dryness,skin_concerns_oiliness,skin_concerns_puffiness,skin_concerns_redness_irritation,five_star,four_star,three_star,two_star,one_star
0,https://www.sephora.com/ca/en/product/protini-...,DRUNK ELEPHANT,Protini™ Polypeptide Moisturizer,1,Moisturizers,Moisturizers,4504,263031,89,1,...,1,1,0,0,1,2612,557,415,416,504
1,https://www.sephora.com/ca/en/product/the-wate...,TATCHA,The Water Cream,1932920,Moisturizers,Moisturizers,3100,237318,89,1,...,1,1,0,0,1,1848,415,314,236,287
2,https://www.sephora.com/ca/en/product/the-dewy...,TATCHA,The Dewy Skin Cream,2181006,Moisturizers,Moisturizers,1475,119147,89,1,...,1,1,0,0,1,982,158,116,117,102
3,https://www.sephora.com/ca/en/product/creme-de...,LA MER,Crème de la Mer Moisturizer,1932201,Moisturizers,Moisturizers,663,75093,235,0,...,0,1,0,0,0,390,78,54,54,87
4,https://www.sephora.com/ca/en/product/facial-t...,SK-II,Facial Treatment Essence (Pitera Essence),1850346,Moisturizers,Mists & Essences,985,86094,241,1,...,1,1,0,0,1,548,149,89,76,123
5,https://www.sephora.com/ca/en/product/the-true...,BELIF,The True Cream Aqua Bomb,1686427,Moisturizers,Moisturizers,4798,192787,50,1,...,1,1,0,0,1,3454,727,263,183,171
6,https://www.sephora.com/ca/en/product/drunk-el...,DRUNK ELEPHANT,Lala Retro™ Whipped Moisturizer with Ceramides,1,Moisturizers,Moisturizers,1228,60619,79,1,...,0,1,0,0,1,704,226,128,80,90
7,https://www.sephora.com/ca/en/product/the-ordi...,THE ORDINARY,100% Organic Cold-Pressed Rose Hip Seed Oil,1,Moisturizers,Face Oils,915,140702,9.9,1,...,0,0,0,0,1,658,108,43,44,62
8,https://www.sephora.com/ca/en/product/ultra-fa...,KIEHL'S SINCE 1851,Ultra Facial Cream,2172542,Moisturizers,Moisturizers,1066,100031,22.5,1,...,1,1,0,0,1,686,182,73,68,57
9,https://www.sephora.com/ca/en/product/rose-dee...,FRESH,Rose & Hyaluronic Acid Deep Hydration Moisturizer,1788397,Moisturizers,Moisturizers,1436,100869,55,1,...,1,1,0,0,1,930,182,136,114,74


In [29]:
df.to_csv('Sephora_Table.csv', index=False)